In [1]:
import pandas as pd

In [10]:
data_stk = {
    "Model": [
        "Static NB", "Static LR", "Static DT", "Static RF", "Static XGB", "Static MLP",
        "Slope-based NB", "Slope-based LR", "Slope-based DT", "Slope-based RF", "Slope-based XGB", "Slope-based MLP",
        "Longitudinal NB", "Longitudinal LR", "Longitudinal DT", "Longitudinal RF", "Longitudinal XGB", "Longitudinal MLP"
    ],
    "Sensitivity": [0.581, 0.507, 0.184, 0.199, 0.111, 0.228,
                    0.875, 0.477, 0.206, 0.228, 0.081, 0.206,
                    0.824, 0.338, 0.214, 0.273, 0.096, 0.169],
    "F1": [0.049, 0.064, 0.040, 0.045, 0.049, 0.068,
           0.045, 0.062, 0.042, 0.043, 0.041, 0.061,
           0.048, 0.063, 0.044, 0.052, 0.045, 0.050],
    "F2": [0.107, 0.133, 0.074, 0.084, 0.072, 0.117,
           0.105, 0.130, 0.080, 0.084, 0.057, 0.105,
           0.110, 0.122, 0.084, 0.101, 0.065, 0.086],
    "AUC": [0.567, 0.602, 0.514, 0.530, 0.576, 0.597,
            0.530, 0.607, 0.528, 0.531, 0.589, 0.586,
            0.562, 0.611, 0.561, 0.534, 0.567, 0.591]
}

data_stroke = pd.DataFrame(data_stk)

# --- Rank Fusion (Reciprocal Rank Fusion) ---
metrics = ["F1", "F2", "AUC"]
k = 0 
for m in metrics:
    data_stroke[f"{m}_rank"] = data_stroke[m].rank(ascending=False, method="min")

# Compute reciprocal rank score
data_stroke["RRF_score"] = data_stroke[[f"{m}_rank" for m in metrics]].apply(
    lambda row: sum(1.0 / (k + row[r]) for r in row.index),
    axis=1
)

# Final ranking
data_stroke["Final_rank"] = data_stroke["RRF_score"].rank(ascending=False, method="min")

if data_stroke["RRF_score"].duplicated().any():
    print("duplicate RRF")


data_stroke[["Model", "Sensitivity", "F1", "F2", "AUC", "RRF_score", "Final_rank"]].sort_values("Final_rank")

,Model,Sensitivity,F1,F2,AUC,RRF_score,Final_rank
1,Static LR,0.507,0.064,0.133,0.602,1.833333,1.0
13,Longitudinal LR,0.338,0.063,0.122,0.611,1.666667,2.0
5,Static MLP,0.228,0.068,0.117,0.597,1.500000,3.0
7,Slope-based LR,0.477,0.062,0.130,0.607,1.250000,4.0
11,Slope-based MLP,0.206,0.061,0.105,0.586,0.485714,5.0
17,Longitudinal MLP,0.169,0.050,0.086,0.591,0.442857,6.0
0,Static NB,0.581,0.049,0.107,0.567,0.402778,7.0
12,Longitudinal NB,0.824,0.048,0.110,0.562,0.390909,8.0
15,Longitudinal RF,0.273,0.052,0.101,0.534,0.354701,9.0
4,Static XGB,0.111,0.049,0.072,0.576,0.312500,10.0


In [11]:

data_death = {
    "Model": [
        "Static NB", "Static LR", "Static DT", "Static RF", "Static XGB", "Static MLP",
        "Longitudinal NB", "Longitudinal LR", "Longitudinal DT", "Longitudinal RF", "Longitudinal XGB", "Longitudinal MLP",
        "All-features NB", "All-features LR", "All-features DT", "All-features RF", "All-features XGB", "All-featues MLP"
    ],
    "F1": [0.161, 0.177, 0.162, 0.191, 0.194, 0.172,
           0.131, 0.181, 0.153, 0.183, 0.195, 0.172,
           0.134, 0.180, 0.166, 0.184, 0.192, 0.159],
    "F2": [0.305, 0.331, 0.303, 0.347, 0.348, 0.308,
           0.268, 0.337, 0.294, 0.337, 0.350, 0.308,
           0.272, 0.334, 0.310, 0.339, 0.346, 0.288],
    "AUC": [0.718, 0.774, 0.691, 0.780, 0.777, 0.734,
            0.712, 0.777, 0.695, 0.779, 0.785, 0.731,
            0.708, 0.774, 0.690, 0.753, 0.779, 0.723]
}

df_death = pd.DataFrame(data_death)

# --- Rank Fusion (Reciprocal Rank Fusion) ---
metrics = ["F1", "F2", "AUC"]
k = 0  # reciprocal rank parameter (can change if needed, e.g. 60)

# Rank for each metric (higher is better)
for m in metrics:
    df_death[f"{m}_rank"] = df_death[m].rank(ascending=False, method="min")

# Compute reciprocal rank fusion score
df_death["RRF_score"] = df_death[[f"{m}_rank" for m in metrics]].apply(
    lambda row: sum(1.0 / (k + row[r]) for r in row.index),
    axis=1
)

# Final fused rank
df_death["Final_rank"] = df_death["RRF_score"].rank(ascending=False, method="min")

if df_death["RRF_score"].duplicated().any():
    print("duplicate RRF")

# Show sorted results
df_death[["Model", "F1", "F2", "AUC", "RRF_score", "Final_rank"]].sort_values("Final_rank")


,Model,F1,F2,AUC,RRF_score,Final_rank
10,Longitudinal XGB,0.195,0.350,0.785,3.000000,1.0
4,Static XGB,0.194,0.348,0.777,1.200000,2.0
3,Static RF,0.191,0.347,0.780,1.083333,3.0
16,All-features XGB,0.192,0.346,0.779,0.916667,4.0
9,Longitudinal RF,0.183,0.337,0.779,0.666667,5.0
15,All-features RF,0.184,0.339,0.753,0.511111,6.0
7,Longitudinal LR,0.181,0.337,0.777,0.509524,7.0
13,All-features LR,0.180,0.334,0.774,0.392857,8.0
1,Static LR,0.177,0.331,0.774,0.365079,9.0
5,Static MLP,0.172,0.308,0.734,0.290909,10.0


In [12]:

data_cvdeath = {
    "Model": [
        "Static NB", "Static LR", "Static DT", "Static RF", "Static XGB", "Static MLP",
        "Longitudinal NB", "Longitudinal LR", "Longitudinal DT", "Longitudinal RF", "Longitudinal XGB", "Longitudinal MLP",
        "All-features NB", "All-features LR", "All-features DT", "All-features RF", "All-features XGB", "All-features MLP"
    ],
    "F1": [0.129, 0.146, 0.102, 0.172, 0.177, 0.152,
           0.113, 0.147, 0.108, 0.140, 0.168, 0.148,
           0.115, 0.151, 0.120, 0.173, 0.157, 0.149],
    "F2": [0.248, 0.275, 0.188, 0.280, 0.300, 0.270,
           0.234, 0.279, 0.201, 0.247, 0.282, 0.260,
           0.237, 0.271, 0.218, 0.285, 0.269, 0.265],
    "AUC": [0.729, 0.760, 0.651, 0.774, 0.767, 0.731,
            0.723, 0.766, 0.634, 0.710, 0.766, 0.731,
            0.722, 0.727, 0.641, 0.764, 0.750, 0.734]
}

df_cvdeath = pd.DataFrame(data_cvdeath)

# --- Rank Fusion (Reciprocal Rank Fusion) ---
metrics = ["F1", "F2", "AUC"]
k = 0  # reciprocal rank parameter

# Rank for each metric (higher is better)
for m in metrics:
    df_cvdeath[f"{m}_rank"] = df_cvdeath[m].rank(ascending=False, method="min")

# Compute reciprocal rank fusion score
df_cvdeath["RRF_score"] = df_cvdeath[[f"{m}_rank" for m in metrics]].apply(
    lambda row: sum(1.0 / (k + row[r]) for r in row.index),
    axis=1
)

# Final fused rank
df_cvdeath["Final_rank"] = df_cvdeath["RRF_score"].rank(ascending=False, method="min")

if df_cvdeath["RRF_score"].duplicated().any():
    print("duplicate RRF")

# Show sorted results
df_cvdeath[["Model", "F1", "F2", "AUC", "RRF_score", "Final_rank"]].sort_values("Final_rank")


,Model,F1,F2,AUC,RRF_score,Final_rank
4,Static XGB,0.177,0.300,0.767,2.500000,1.0
3,Static RF,0.172,0.280,0.774,1.583333,2.0
15,All-features RF,0.173,0.285,0.764,1.200000,3.0
10,Longitudinal XGB,0.168,0.282,0.766,0.916667,4.0
7,Longitudinal LR,0.147,0.279,0.766,0.633333,5.0
16,All-features XGB,0.157,0.269,0.750,0.453968,6.0
1,Static LR,0.146,0.275,0.760,0.424242,7.0
5,Static MLP,0.152,0.270,0.731,0.402778,8.0
13,All-features LR,0.151,0.271,0.727,0.369048,9.0
17,All-features MLP,0.149,0.265,0.734,0.350000,10.0


In [13]:
import pandas as pd

# Data
data_hf = {
    "Model": [
        "Static NB", "Static LR", "Static DT", "Static RF", "Static XGB", "Static MLP",
        "Slope-based NB", "Slope-based LR", "Slope-based DT", "Slope-based RF", "Slope-based XGB", "Slope-based MLP",
        "Longitudinal NB", "Longitudinal LR", "Longitudinal DT", "Longitudinal RF", "Longitudinal XGB", "Longitudinal MLP"
    ],
    "F1": [0.331, 0.357, 0.381, 0.386, 0.412, 0.335,
           0.326, 0.377, 0.389, 0.371, 0.388, 0.341,
           0.326, 0.368, 0.385, 0.369, 0.404, 0.348],
    "F2": [0.482, 0.542, 0.531, 0.559, 0.570, 0.479,
           0.496, 0.548, 0.537, 0.547, 0.554, 0.487,
           0.498, 0.547, 0.538, 0.546, 0.567, 0.501],
    "AUC": [0.723, 0.772, 0.755, 0.785, 0.806, 0.728,
            0.719, 0.785, 0.760, 0.774, 0.796, 0.731,
            0.716, 0.782, 0.762, 0.758, 0.801, 0.739]
}

df_hf = pd.DataFrame(data_hf)

# --- Rank Fusion (Reciprocal Rank Fusion) ---
metrics = ["F1", "F2", "AUC"]
k = 0  # reciprocal rank parameter

# Rank each metric (higher is better)
for m in metrics:
    df_hf[f"{m}_rank"] = df_hf[m].rank(ascending=False, method="min")

# Compute reciprocal rank fusion score
df_hf["RRF_score"] = df_hf[[f"{m}_rank" for m in metrics]].apply(
    lambda row: sum(1.0 / (k + row[r]) for r in row.index),
    axis=1
)

# Final fused rank
df_hf["Final_rank"] = df_hf["RRF_score"].rank(ascending=False, method="min")

if df_hf["RRF_score"].duplicated().any():
    print("duplicate RRF")

# Show sorted results
df_hf[["Model", "F1", "F2", "AUC", "RRF_score", "Final_rank"]].sort_values("Model")


,Model,F1,F2,AUC,RRF_score,Final_rank
14,Longitudinal DT,0.385,0.538,0.762,0.377778,9.0
13,Longitudinal LR,0.368,0.547,0.782,0.424242,7.0
17,Longitudinal MLP,0.348,0.501,0.739,0.230769,13.0
12,Longitudinal NB,0.326,0.498,0.716,0.185808,16.0
15,Longitudinal RF,0.369,0.546,0.758,0.315909,11.0
16,Longitudinal XGB,0.404,0.567,0.801,1.500000,2.0
8,Slope-based DT,0.389,0.537,0.760,0.524242,6.0
7,Slope-based LR,0.377,0.548,0.785,0.575000,5.0
11,Slope-based MLP,0.341,0.487,0.731,0.205357,14.0
6,Slope-based NB,0.326,0.496,0.719,0.184314,17.0


In [17]:

data_inp = {
    "Model": [
        "Static NB", "Static LR", "Static DT", "Static RF", "Static XGB", "Static MLP",
        "Slope-based NB", "Slope-based LR", "Slope-based DT", "Slope-based RF", "Slope-based XGB", "Slope-based MLP",
        "Longitudinal NB", "Longitudinal LR", "Longitudinal DT", "Longitudinal RF", "Longitudinal XGB", "Longitudinal MLP"
    ],
    "F1": [0.399, 0.430, 0.407, 0.409, 0.418, 0.376,
           0.377, 0.428, 0.391, 0.409, 0.420, 0.379,
           0.376, 0.425, 0.410, 0.409, 0.425, 0.383],
    "F2": [0.552, 0.548, 0.501, 0.587, 0.589, 0.476,
           0.571, 0.552, 0.477, 0.587, 0.590, 0.480,
           0.570, 0.553, 0.505, 0.587, 0.588, 0.484],
    "AUC": [0.646, 0.683, 0.648, 0.627, 0.632, 0.623,
            0.639, 0.679, 0.631, 0.627, 0.670, 0.619,
            0.638, 0.678, 0.649, 0.619, 0.670, 0.630]
}

df_inp = pd.DataFrame(data_inp)

# --- Rank Fusion (Reciprocal Rank Fusion) ---
metrics = ["F1", "F2", "AUC"]
k = 0  # reciprocal rank parameter

# Rank each metric (higher is better)
for m in metrics:
    df_inp[f"{m}_rank"] = df_inp[m].rank(ascending=False, method="min")

# Compute reciprocal rank fusion score
df_inp["RRF_score"] = df_inp[[f"{m}_rank" for m in metrics]].apply(
    lambda row: sum(1.0 / (k + row[r]) for r in row.index),
    axis=1
)

# Final fused rank
df_inp["Final_rank"] = df_inp["RRF_score"].rank(ascending=False, method="min")

if df_inp["RRF_score"].duplicated().any():
    print("duplicate RRF")

# Show sorted results
df_inp[["Model", "F1", "F2", "AUC", "RRF_score", "Final_rank"]].sort_values("Final_rank")


duplicate RRF


,Model,F1,F2,AUC,RRF_score,Final_rank
1,Static LR,0.430,0.548,0.683,2.083333,1.0
10,Slope-based XGB,0.420,0.590,0.670,1.450000,2.0
7,Slope-based LR,0.428,0.552,0.679,1.100000,3.0
16,Longitudinal XGB,0.425,0.588,0.670,0.916667,4.0
13,Longitudinal LR,0.425,0.553,0.678,0.777778,5.0
4,Static XGB,0.418,0.589,0.632,0.757576,6.0
3,Static RF,0.409,0.587,0.627,0.446429,7.0
9,Slope-based RF,0.409,0.587,0.627,0.446429,7.0
15,Longitudinal RF,0.409,0.587,0.619,0.433824,9.0
14,Longitudinal DT,0.410,0.505,0.649,0.386447,10.0


In [18]:

data_acs = {
    "Model": [
        "Static NB", "Static LR", "Static DT", "Static RF", "Static XGB", "Static MLP",
        "Slope Based NB", "Slope Based LR", "Slope Based DT", "Slope Based RF", "Slope Based XGB", "Slope Based MLP",
        "Longitudinal NB", "Longitudinal LR", "Longitudinal DT", "Longitudinal RF", "Longitudinal XGB", "Longitudinal MLP"
    ],
    "F1": [0.064, 0.054, 0.055, 0.053, 0.073, 0.088,
           0.054, 0.056, 0.072, 0.058, 0.086, 0.084,
           0.054, 0.055, 0.053, 0.044, 0.103, 0.092],
    "F2": [0.135, 0.118, 0.097, 0.096, 0.101, 0.139,
           0.118, 0.122, 0.133, 0.104, 0.111, 0.130,
           0.119, 0.121, 0.097, 0.082, 0.138, 0.141],
    "AUC": [0.738, 0.740, 0.596, 0.605, 0.721, 0.692,
            0.721, 0.736, 0.620, 0.623, 0.691, 0.687,
            0.720, 0.737, 0.604, 0.580, 0.715, 0.698]
}

df_acs = pd.DataFrame(data_acs)

# --- Rank Fusion (Reciprocal Rank Fusion) ---
metrics = ["F1", "F2", "AUC"]
k = 0  # reciprocal rank parameter

# Rank each metric (higher is better)
for m in metrics:
    df_acs[f"{m}_rank"] = df_acs[m].rank(ascending=False, method="min")

# Compute reciprocal rank fusion score
df_acs["RRF_score"] = df_acs[[f"{m}_rank" for m in metrics]].apply(
    lambda row: sum(1.0 / (k + row[r]) for r in row.index),
    axis=1
)

# Final fused rank
df_acs["Final_rank"] = df_acs["RRF_score"].rank(ascending=False, method="min")

if df_acs["RRF_score"].duplicated().any():
    print("duplicate RRF")

# Show sorted results
#df_acs[["Model", "F1", "F2", "AUC", "Final_rank", "RRF_score"]].sort_values("Final_rank")
df_acs.sort_values("Final_rank").head()


,Model,F1,F2,AUC,F1_rank,F2_rank,AUC_rank,RRF_score,Final_rank
17,Longitudinal MLP,0.092,0.141,0.698,2.0,1.0,9.0,1.611111,1.0
16,Longitudinal XGB,0.103,0.138,0.715,1.0,3.0,8.0,1.458333,2.0
1,Static LR,0.054,0.118,0.740,13.0,10.0,1.0,1.176923,3.0
5,Static MLP,0.088,0.139,0.692,3.0,2.0,10.0,0.933333,4.0
0,Static NB,0.064,0.135,0.738,8.0,4.0,2.0,0.875000,5.0


In [20]:
data_stk_new = {
    "Model": [
        "Static NB", "Static LR", "Static DT", "Static RF", "Static XGB", "Static MLP",
        "Slope-based NB", "Slope-based LR", "Slope-based DT", "Slope-based RF", "Slope-based XGB", "Slope-based MLP",
        "Longitudinal NB", "Longitudinal LR", "Longitudinal DT", "Longitudinal RF", "Longitudinal XGB", "Longitudinal MLP"
    ],
    "F1": [0.023, 0.034, 0.020, 0.022, 0.000, 0.010,
           0.023, 0.033, 0.029, 0.042, 0.000, 0.024,
           0.023, 0.047, 0.035, 0.025, 0.000, 0.037],
    "F2": [0.056, 0.072, 0.039, 0.041, 0.000, 0.013,
           0.056, 0.071, 0.054, 0.079, 0.000, 0.037,
           0.056, 0.098, 0.064, 0.047, 0.000, 0.052],
    "AUC": [0.624, 0.650, 0.554, 0.496, 0.561, 0.614,
            0.597, 0.637, 0.548, 0.571, 0.588, 0.628,
            0.581, 0.664, 0.575, 0.535, 0.611, 0.641],

}

df_stk = pd.DataFrame(data_stk_new)

# --- Rank Fusion (Reciprocal Rank Fusion) ---
metrics = ["F1", "F2", "AUC"]
k = 0  # reciprocal rank parameter

# Rank each metric (higher is better)
for m in metrics:
    df_stk[f"{m}_rank"] = df_stk[m].rank(ascending=False, method="min")

# Compute reciprocal rank fusion score
df_stk["RRF_score"] = df_stk[[f"{m}_rank" for m in metrics]].apply(
    lambda row: sum(1.0 / (k + row[r]) for r in row.index),
    axis=1
)

# Final fused rank
df_stk["Final_rank"] = df_stk["RRF_score"].rank(ascending=False, method="min")

if df_stk["RRF_score"].duplicated().any():
    print("duplicate RRF")

# Show sorted results
df_stk[["Model", "F1", "F2", "AUC", "Final_rank", "RRF_score"]].sort_values("Model")


,Model,F1,F2,AUC,Final_rank,RRF_score
14,Longitudinal DT,0.035,0.064,0.575,6.0,0.533333
13,Longitudinal LR,0.047,0.098,0.664,1.0,3.000000
17,Longitudinal MLP,0.037,0.052,0.641,4.0,0.766667
12,Longitudinal NB,0.023,0.056,0.581,10.0,0.357576
15,Longitudinal RF,0.025,0.047,0.535,13.0,0.274733
16,Longitudinal XGB,0.000,0.000,0.611,14.0,0.250000
8,Slope-based DT,0.029,0.054,0.548,11.0,0.316468
7,Slope-based LR,0.033,0.071,0.637,5.0,0.666667
11,Slope-based MLP,0.024,0.037,0.628,8.0,0.382540
6,Slope-based NB,0.023,0.056,0.597,9.0,0.377778


## NEW VERSION

In [8]:
data = {
    "Model": [
        "Static NB", "Static LR", "Static DT", "Static RF", "Static XGB", "Static MLP",
        "Slope Based NB", "Slope Based LR", "Slope Based DT", "Slope Based RF", "Slope Based XGB", "Slope Based MLP",
        "Longitudinal NB", "Longitudinal LR", "Longitudinal DT", "Longitudinal RF", "Longitudinal XGB", "Longitudinal MLP"
    ],
    "F1": [
        0.066, 0.060, 0.050, 0.048, 0.077, 0.091,
        0.045, 0.060, 0.042, 0.042, 0.091, 0.073,
        0.054, 0.055, 0.053, 0.044, 0.103, 0.092
    ],
    "F2": [
        0.136, 0.127, 0.085, 0.084, 0.102, 0.143,
        0.100, 0.128, 0.080, 0.075, 0.124, 0.117,
        0.119, 0.121, 0.097, 0.082, 0.138, 0.141
    ],
    "AUC": [
        0.737, 0.734, 0.609, 0.604, 0.703, 0.678,
        0.712, 0.729, 0.544, 0.597, 0.691, 0.661,
        0.720, 0.737, 0.604, 0.580, 0.715, 0.698
    ]
}



df = pd.DataFrame(data)

# --- Rank Fusion (Reciprocal Rank Fusion) ---
metrics = ["F1", "F2", "AUC"]
k = 0  # reciprocal rank parameter

# Rank each metric (higher is better)
for m in metrics:
    df[f"{m}_rank"] = df[m].rank(ascending=False, method="min")

# Compute reciprocal rank fusion score
df["RRF_score"] = df[[f"{m}_rank" for m in metrics]].apply(
    lambda row: sum(1.0 / (k + row[r]) for r in row.index),
    axis=1
)

# Final fused rank
df["Final_rank"] = df["RRF_score"].rank(ascending=False, method="min")

if df["RRF_score"].duplicated().any():
    print("duplicate RRF")

# Show sorted results
df[["Model", "F1", "F2", "AUC", "Final_rank", "RRF_score"]].sort_values("Model")


,Model,F1,F2,AUC,Final_rank,RRF_score
14,Longitudinal DT,0.053,0.097,0.604,13.0,0.231685
13,Longitudinal LR,0.055,0.121,0.737,4.0,1.225000
17,Longitudinal MLP,0.092,0.141,0.698,5.0,1.111111
12,Longitudinal NB,0.054,0.119,0.720,10.0,0.402020
15,Longitudinal RF,0.044,0.082,0.580,16.0,0.183824
16,Longitudinal XGB,0.103,0.138,0.715,1.0,1.500000
8,Slope Based DT,0.042,0.080,0.544,18.0,0.173203
7,Slope Based LR,0.060,0.128,0.729,8.0,0.575000
11,Slope Based MLP,0.073,0.117,0.661,11.0,0.350000
6,Slope Based NB,0.045,0.100,0.712,12.0,0.292857


In [9]:
data = {
    "Model": [
        "Static NB", "Static LR", "Static DT", "Static RF", "Static XGB", "Static MLP",
        "Slope-based NB", "Slope-based LR", "Slope-based DT", "Slope-based RF", "Slope-based XGB", "Slope-based MLP",
        "Longitudinal NB", "Longitudinal LR", "Longitudinal DT", "Longitudinal RF", "Longitudinal XGB", "Longitudinal MLP"
    ],
    "F1": [
        0.123, 0.145, 0.121, 0.133, 0.153, 0.138,
        0.095, 0.143, 0.117, 0.133, 0.150, 0.132,
        0.115, 0.151, 0.120, 0.173, 0.157, 0.149
    ],
    "F2": [
        0.228, 0.285, 0.225, 0.235, 0.261, 0.249,
        0.204, 0.284, 0.217, 0.235, 0.256, 0.239,
        0.237, 0.271, 0.218, 0.285, 0.269, 0.265
    ],
    "AUC": [
        0.715, 0.774, 0.651, 0.707, 0.757, 0.712,
        0.706, 0.776, 0.650, 0.684, 0.753, 0.701,
        0.722, 0.727, 0.641, 0.764, 0.750, 0.734
    ]
}


df = pd.DataFrame(data)

# --- Rank Fusion (Reciprocal Rank Fusion) ---
metrics = ["F1", "F2", "AUC"]
k = 0  # reciprocal rank parameter

# Rank each metric (higher is better)
for m in metrics:
    df[f"{m}_rank"] = df[m].rank(ascending=False, method="min")

# Compute reciprocal rank fusion score
df["RRF_score"] = df[[f"{m}_rank" for m in metrics]].apply(
    lambda row: sum(1.0 / (k + row[r]) for r in row.index),
    axis=1
)

# Final fused rank
df["Final_rank"] = df["RRF_score"].rank(ascending=False, method="min")

if df["RRF_score"].duplicated().any():
    print("duplicate RRF")

# Show sorted results
df[["Model", "F1", "F2", "AUC", "Final_rank", "RRF_score"]].sort_values("Model")


,Model,F1,F2,AUC,Final_rank,RRF_score
14,Longitudinal DT,0.120,0.218,0.641,17.0,0.184722
13,Longitudinal LR,0.151,0.271,0.727,6.0,0.625000
17,Longitudinal MLP,0.149,0.265,0.734,8.0,0.476190
12,Longitudinal NB,0.115,0.237,0.722,11.0,0.260844
15,Longitudinal RF,0.173,0.285,0.764,1.0,2.333333
16,Longitudinal XGB,0.157,0.269,0.750,4.0,0.866667
8,Slope-based DT,0.117,0.217,0.650,18.0,0.180147
7,Slope-based LR,0.143,0.284,0.776,3.0,1.458333
11,Slope-based MLP,0.132,0.239,0.701,12.0,0.254762
6,Slope-based NB,0.095,0.204,0.706,16.0,0.188034


In [10]:
data = {
    "Model": [
        "Static NB", "Static LR", "Static DT", "Static RF", "Static XGB", "Static MLP",
        "Slope-based NB", "Slope-based LR", "Slope-based DT", "Slope-based RF", "Slope-based XGB", "Slope-based MLP",
        "Longitudinal NB", "Longitudinal LR", "Longitudinal DT", "Longitudinal RF", "Longitudinal XGB", "Longitudinal MLP"
    ],
    "F1": [
        0.161, 0.171, 0.162, 0.180, 0.176, 0.168,
        0.111, 0.170, 0.151, 0.175, 0.179, 0.164,
        0.134, 0.180, 0.166, 0.184, 0.192, 0.159
    ],
    "F2": [
        0.295, 0.325, 0.303, 0.332, 0.322, 0.306,
        0.234, 0.325, 0.288, 0.324, 0.329, 0.298,
        0.272, 0.334, 0.310, 0.339, 0.346, 0.288
    ],
    "AUC": [
        0.705, 0.774, 0.691, 0.769, 0.762, 0.727,
        0.694, 0.772, 0.686, 0.767, 0.755, 0.716,
        0.708, 0.774, 0.690, 0.753, 0.779, 0.723
    ]
}

df = pd.DataFrame(data)

# --- Rank Fusion (Reciprocal Rank Fusion) ---
metrics = ["F1", "F2", "AUC"]
k = 0  # reciprocal rank parameter

# Rank each metric (higher is better)
for m in metrics:
    df[f"{m}_rank"] = df[m].rank(ascending=False, method="min")

# Compute reciprocal rank fusion score
df["RRF_score"] = df[[f"{m}_rank" for m in metrics]].apply(
    lambda row: sum(1.0 / (k + row[r]) for r in row.index),
    axis=1
)

# Final fused rank
df["Final_rank"] = df["RRF_score"].rank(ascending=False, method="min")

if df["RRF_score"].duplicated().any():
    print("duplicate RRF")

# Show sorted results
df[["Model", "F1", "F2", "AUC", "Final_rank", "RRF_score"]].sort_values("Model")

,Model,F1,F2,AUC,Final_rank,RRF_score
14,Longitudinal DT,0.166,0.310,0.690,11.0,0.249733
13,Longitudinal LR,0.180,0.334,0.774,2.0,1.166667
17,Longitudinal MLP,0.159,0.288,0.723,13.0,0.224242
12,Longitudinal NB,0.134,0.272,0.708,16.0,0.194570
15,Longitudinal RF,0.184,0.339,0.753,3.0,1.111111
16,Longitudinal XGB,0.192,0.346,0.779,1.0,3.000000
8,Slope-based DT,0.151,0.288,0.686,17.0,0.184722
7,Slope-based LR,0.170,0.325,0.772,6.0,0.527778
11,Slope-based MLP,0.164,0.298,0.716,12.0,0.243590
6,Slope-based NB,0.111,0.234,0.694,18.0,0.177778


In [11]:
data = {
    "Model": [
        "Static NB", "Static LR", "Static DT", "Static RF", "Static XGB", "Static MLP",
        "Slope-based NB", "Slope-based LR", "Slope-based DT", "Slope-based RF", "Slope-based XGB", "Slope-based MLP",
        "Longitudinal NB", "Longitudinal LR", "Longitudinal DT", "Longitudinal RF", "Longitudinal XGB", "Longitudinal MLP"
    ],
    "F1": [
        0.330, 0.371, 0.342, 0.358, 0.372, 0.328,
        0.309, 0.370, 0.346, 0.358, 0.366, 0.322,
        0.326, 0.368, 0.385, 0.369, 0.404, 0.348
    ],
    "F2": [
        0.434, 0.534, 0.483, 0.528, 0.535, 0.469,
        0.484, 0.531, 0.488, 0.524, 0.531, 0.470,
        0.498, 0.547, 0.538, 0.546, 0.567, 0.501
    ],
    "AUC": [
        0.721, 0.775, 0.718, 0.747, 0.766, 0.712,
        0.714, 0.774, 0.713, 0.764, 0.751, 0.714,
        0.716, 0.782, 0.762, 0.758, 0.801, 0.739
    ]
}

df = pd.DataFrame(data)

# --- Rank Fusion (Reciprocal Rank Fusion) ---
metrics = ["F1", "F2", "AUC"]
k = 0  # reciprocal rank parameter

# Rank each metric (higher is better)
for m in metrics:
    df[f"{m}_rank"] = df[m].rank(ascending=False, method="min")

# Compute reciprocal rank fusion score
df["RRF_score"] = df[[f"{m}_rank" for m in metrics]].apply(
    lambda row: sum(1.0 / (k + row[r]) for r in row.index),
    axis=1
)

# Final fused rank
df["Final_rank"] = df["RRF_score"].rank(ascending=False, method="min")

if df["RRF_score"].duplicated().any():
    print("duplicate RRF")

# Show sorted results
df[["Model", "F1", "F2", "AUC", "Final_rank", "RRF_score"]].sort_values("Model")

,Model,F1,F2,AUC,Final_rank,RRF_score
14,Longitudinal DT,0.385,0.538,0.762,3.0,0.892857
13,Longitudinal LR,0.368,0.547,0.782,2.0,1.142857
17,Longitudinal MLP,0.348,0.501,0.739,11.0,0.272727
12,Longitudinal NB,0.326,0.498,0.716,14.0,0.217262
15,Longitudinal RF,0.369,0.546,0.758,6.0,0.625000
16,Longitudinal XGB,0.404,0.567,0.801,1.0,3.000000
8,Slope-based DT,0.346,0.488,0.713,13.0,0.219080
7,Slope-based LR,0.370,0.531,0.774,7.0,0.592857
11,Slope-based MLP,0.322,0.470,0.714,17.0,0.187990
6,Slope-based NB,0.309,0.484,0.714,16.0,0.193651


In [12]:
data = {
    "Model": [
        "Static NB", "Static LR", "Static DT", "Static RF", "Static XGB", "Static MLP",
        "Slope-based NB", "Slope-based LR", "Slope-based DT", "Slope-based RF", "Slope-based XGB", "Slope-based MLP",
        "Longitudinal NB", "Longitudinal LR", "Longitudinal DT", "Longitudinal RF", "Longitudinal XGB", "Longitudinal MLP"
    ],
    "F1": [
        0.389, 0.407, 0.379, 0.412, 0.419, 0.375,
        0.367, 0.417, 0.375, 0.426, 0.423, 0.370,
        0.376, 0.425, 0.410, 0.409, 0.425, 0.383
    ],
    "F2": [
        0.483, 0.530, 0.463, 0.530, 0.528, 0.479,
        0.574, 0.539, 0.455, 0.552, 0.534, 0.472,
        0.570, 0.553, 0.505, 0.587, 0.588, 0.484
    ],
    "AUC": [
        0.652, 0.663, 0.611, 0.666, 0.677, 0.616,
        0.632, 0.668, 0.609, 0.680, 0.674, 0.614,
        0.638, 0.678, 0.649, 0.619, 0.670, 0.630
    ]
}

df = pd.DataFrame(data)

# --- Rank Fusion (Reciprocal Rank Fusion) ---
metrics = ["F1", "F2", "AUC"]
k = 0  # reciprocal rank parameter

# Rank each metric (higher is better)
for m in metrics:
    df[f"{m}_rank"] = df[m].rank(ascending=False, method="min")

# Compute reciprocal rank fusion score
df["RRF_score"] = df[[f"{m}_rank" for m in metrics]].apply(
    lambda row: sum(1.0 / (k + row[r]) for r in row.index),
    axis=1
)

# Final fused rank
df["Final_rank"] = df["RRF_score"].rank(ascending=False, method="min")

if df["RRF_score"].duplicated().any():
    print("duplicate RRF")

# Show sorted results
df[["Model", "F1", "F2", "AUC", "Final_rank", "RRF_score"]].sort_values("Model")

,Model,F1,F2,AUC,Final_rank,RRF_score
14,Longitudinal DT,0.410,0.505,0.649,12.0,0.308333
13,Longitudinal LR,0.425,0.553,0.678,3.0,1.200000
17,Longitudinal MLP,0.383,0.484,0.630,14.0,0.237179
12,Longitudinal NB,0.376,0.570,0.638,9.0,0.412338
15,Longitudinal RF,0.409,0.587,0.619,4.0,0.682540
16,Longitudinal XGB,0.425,0.588,0.670,2.0,1.700000
8,Slope-based DT,0.375,0.455,0.609,18.0,0.177778
7,Slope-based LR,0.417,0.539,0.668,7.0,0.476190
11,Slope-based MLP,0.370,0.472,0.614,17.0,0.183824
6,Slope-based NB,0.367,0.574,0.632,8.0,0.472222


In [13]:
data = {
    "Model": [
        "Static NB", "Static LR", "Static DT", "Static RF", "Static XGB", "Static MLP",
        "Slope-based NB", "Slope-based LR", "Slope-based DT", "Slope-based RF", "Slope-based XGB", "Slope-based MLP",
        "Longitudinal NB", "Longitudinal LR", "Longitudinal DT", "Longitudinal RF", "Longitudinal XGB", "Longitudinal MLP"
    ],
    "F1": [
        0.045, 0.056, 0.038, 0.029, 0.056, 0.061,
        0.044, 0.065, 0.042, 0.047, 0.052, 0.060,
        0.048, 0.063, 0.044, 0.052, 0.045, 0.050
    ],
    "F2": [
        0.100, 0.125, 0.074, 0.057, 0.080, 0.106,
        0.103, 0.140, 0.082, 0.091, 0.080, 0.104,
        0.110, 0.122, 0.084, 0.101, 0.065, 0.086
    ],
    "AUC": [
        0.570, 0.605, 0.515, 0.478, 0.576, 0.574,
        0.529, 0.634, 0.503, 0.538, 0.574, 0.575,
        0.562, 0.611, 0.561, 0.534, 0.567, 0.591
    ]
}

df = pd.DataFrame(data)

# --- Rank Fusion (Reciprocal Rank Fusion) ---
metrics = ["F1", "F2", "AUC"]
k = 0  # reciprocal rank parameter

# Rank each metric (higher is better)
for m in metrics:
    df[f"{m}_rank"] = df[m].rank(ascending=False, method="min")

# Compute reciprocal rank fusion score
df["RRF_score"] = df[[f"{m}_rank" for m in metrics]].apply(
    lambda row: sum(1.0 / (k + row[r]) for r in row.index),
    axis=1
)

# Final fused rank
df["Final_rank"] = df["RRF_score"].rank(ascending=False, method="min")

if df["RRF_score"].duplicated().any():
    print("duplicate RRF")

# Show sorted results
df[["Model", "F1", "F2", "AUC", "Final_rank", "RRF_score"]].sort_values("Model")

,Model,F1,F2,AUC,Final_rank,RRF_score
14,Longitudinal DT,0.044,0.084,0.561,15.0,0.238095
13,Longitudinal LR,0.063,0.122,0.611,2.0,1.333333
17,Longitudinal MLP,0.050,0.086,0.591,7.0,0.452020
12,Longitudinal NB,0.048,0.110,0.562,8.0,0.440909
15,Longitudinal RF,0.052,0.101,0.534,10.0,0.339286
16,Longitudinal XGB,0.045,0.065,0.567,14.0,0.242157
8,Slope-based DT,0.042,0.082,0.503,16.0,0.198247
7,Slope-based LR,0.065,0.140,0.634,1.0,3.000000
11,Slope-based MLP,0.060,0.104,0.575,5.0,0.583333
6,Slope-based NB,0.044,0.103,0.529,12.0,0.280952
